In [1]:
import os
import sys
from scipy.io import loadmat
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import zipfile
import requests
import shutil
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from deepverse import ParameterManager, Dataset

# DeepVerse generate dataset

In [3]:
# path to the configuration file
config_path = f'scenarios/DT31/param/config.m'

# initialize ParameterManager and load parameters
param_manager = ParameterManager(config_path)
param_manager.params["scenes"] = list(range(100))
param_manager.params["radar"]["enable"] = False
param_manager.params["comm"]["enable"] = True
param_manager.params["camera"] = False
param_manager.params["lidar"] = False
param_manager.params["position"] = True

# generate a dataset
dataset = Dataset(param_manager)

Generating mobility dataset: ⏳ In progress
Generating mobility dataset: ✅ Completed (0.00s)
Generating comm dataset: ⏳ In progress


Reading ray-tracing:   0%|                                                                        | 0/1 [00:00<?, ?it/s]
                                                                                                                        
Generating BS1-UE channels:   0%|                                                                 | 0/1 [00:00<?, ?it/s]
                                                                                                                        
Reading ray-tracing:   0%|                                                                        | 0/1 [00:00<?, ?it/s]
                                                                                                                        
Generating BS1-BS channels:   0%|                                                                 | 0/1 [00:00<?, ?it/s]
                                                                                                                        
Reading ray-tracing:   0%|      

Generating comm dataset: ✅ Completed (2.69s)


In [5]:
def beam_steering_codebook(angles, num_z, num_x):
    d = 0.5
    k_z = np.arange(num_z)
    k_x = np.arange(num_x)
    
    codebook = []
    
    for beam_idx in range(angles.shape[0]):
        z_angle = angles[beam_idx, 0]
        x_angle = angles[beam_idx, 1]
        bf_vector_z = np.exp(1j * 2 * np.pi * k_z * d * np.cos(np.radians(z_angle)))
        bf_vector_x = np.exp(1j * 2 * np.pi * k_x * d * np.cos(np.radians(x_angle)))
        bf_vector = np.outer(bf_vector_z, bf_vector_x).flatten()
        codebook.append(bf_vector)
        
    return np.stack(codebook, axis=0)

# construct beam steering codebook
x_angles = loadmat(f'scenarios/DT31/param/beam_angles.mat')['beam_angles'].squeeze() * 180 / np.pi
num_angles = x_angles.size

# x_angles = np.flip(x_angles)
z_angles = np.full(num_angles, 90)
beam_angles = np.column_stack((z_angles, x_angles))
codebook =  beam_steering_codebook(beam_angles, 1, 16)

FileNotFoundError: [Errno 2] No such file or directory: 'scenarios/DT31/param/beam_angles.mat'